In [1]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import numpy as np
np.set_printoptions(threshold=np.inf)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:52:09) 
[Clang 14.0.6 ]
2.12.0


In [ ]:
"""
(1) 全连接层创建： tf.keras.Sequential()，tf.keras.layers.Dense()
(2) 输出方式： tf.sigmoid()，tf.softmax()，tf.tanh()，tf.nn.relu()，tf.nn.leaky_relu()

1. 全连接层
全连接层在整个网络卷积神经网络中起到特征提取器的作用。全连接层将学到的特征表示映射到样本的标记空间。

（1）在全连接层中创建一层： tf.keras.layers.Dense()
units： 正整数，输出空间的维数
activation=None： 激活函数，不指定则没有
use_bias=True： 布尔值，是否使用偏移向量
kernel_initializer='glorot_uniform'： 核权重矩阵的初始值设定项
bias_initializer='zeros'： 偏差向量的初始值设定项
kernel_regularizer=None： 正则化函数应用于核权矩阵
bias_regularizer=None： 应用于偏差向量的正则化函数
kernel_constraint=None： 对权重矩阵约束
bias_constraint=None： 对偏置向量约束

（2）堆层构建全连接层： tf.keras.Sequential([layer1,layer2,layer3])
通过组合层来构建模型，直接堆叠各个层
"""

In [2]:
"""
1.1 创建单个连接层
首先构造一个shape为[4,478]的随机正态分布，构造一个连接层tf.keras.layers.Dense()，输出512个特征，shape为[4,512]。
将输入层放入连接层自动调用build()函数创建权重w和偏置b，将计算结果返回给out。
"""
# 创建单个连接层
import tensorflow as tf
# 构建输入
x = tf.random.normal([4,478])
# 输入的shape不需要指定，可以自动的根据输入的特征生成对应的权值
net = tf.keras.layers.Dense(512)
# 输出结果
out = net(x)
out.shape  # 输出结果的shape[4,512]
net.kernel.shape # 权值W的形状，W的shape为[478,512]
net.bias.shape # 偏置b的形状，X的shape为[4,478]
# 调用net函数时，自动判断，如果当前没有创建w和b，自动调用build函数

TensorShape([512])

In [3]:
"""
1.2 创建多个连接层
使用tf.keras.Sequential()堆叠构建全连接层；使用tf.keras.layers.Dense()创建每一个全连接层；使用build()构建输入层。
网络结构为[b,10]=>[b,16]=>[b,32]=>[b,8]；通过summary()查看整体网络架构。通过model.trainable_variables查看网络的所有的权重和偏置。

整体网络构架如下：对于第一层，w的shape为[10,16]，b的shape为[16]，共10*16+16=176个参数 ，下面几层的参数个数同理
"""
#（4）创建多个全连接层
# 将所有连接层组成一个列表放入
from tensorflow import keras
# 构建全连接层
model = keras.Sequential([
    keras.layers.Dense(16, activation='relu'), #w的shape为[10,16]，b的shape为[16]，共10*16+16=176个参数  
    keras.layers.Dense(32, activation='relu'), #w的shape为[16,32]，b的shape为[32]
    keras.layers.Dense(8)])  #w的shape为[32,8]，b的shape为[8]
 
model.build(input_shape=[None,10]) #输入层10个特征
model.summary()  # 查看网络结构
# 查看网络所有的权重w和偏置b
for p in model.trainable_variables:
    print(p.name,p.shape)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 16)                176       
                                                                 
 dense_2 (Dense)             (None, 32)                544       
                                                                 
 dense_3 (Dense)             (None, 8)                 264       
                                                                 
Total params: 984
Trainable params: 984
Non-trainable params: 0
_________________________________________________________________
dense_1/kernel:0 (10, 16)
dense_1/bias:0 (16,)
dense_2/kernel:0 (16, 32)
dense_2/bias:0 (32,)
dense_3/kernel:0 (32, 8)
dense_3/bias:0 (8,)


In [5]:
"""
2. 输出映射方式
2.1 sigmoid函数
sigmoid函数： tf.sigmoid(tensor)
把输出结果的范围映射到(0,1)之间，计算公式：\frac{1}{1+e^{-x}}

sigmoid只能保证单个数值的范围在0-1之间
"""
a = tf.linspace(-6,6,10) #-6到6的长度为10的向量
print(a)
print('=================================================================================================================================')
# sigmoid函数将其映射到0-1
print(tf.sigmoid(a))
# 输入数据：array([-6.        , -4.66666667, -3.33333333, -2.        , -0.66666667,0.66666667,  2.        ,  3.33333333,  4.66666667,  6.   
# 输出结果：array([0.00247262, 0.00931596, 0.0344452 , 0.11920292, 0.33924363,0.66075637, 0.88079708, 0.9655548 , 0.99068404, 0.99752738])

tf.Tensor(
[-6.         -4.66666667 -3.33333333 -2.         -0.66666667  0.66666667
  2.          3.33333333  4.66666667  6.        ], shape=(10,), dtype=float64)
tf.Tensor(
[0.00247262 0.00931596 0.0344452  0.11920292 0.33924363 0.66075637
 0.88079708 0.9655548  0.99068404 0.99752738], shape=(10,), dtype=float64)


In [9]:
"""
2.2 softmax函数
对于分类问题，需要每一个概率在0-1之间，且总的概率和为1。
softmax函数：tf.softmax(logits, axis)
logits：一个非空的Tensor。必须是下列类型之一：half， float32，float64。
axis：将在其上执行维度softmax。默认值为-1，表示最后一个维度。
公式为：\sigma (z)_{j}=\frac{e^zj}{\sum_{k=1}^{K}e^{z}k} \;\;\;\; for\;\;j=1,...,K

一般将没有加激活函数的称为Logits，加了softmax后称为Probabilities，经过softmax后，有把最大值放大的过程，相当于把强的变得更强，把弱的变得更弱。
"""
logits = tf.random.uniform([1,10],minval=-2,maxval=2)
print(logits)
print('=================================================================================================================================')

# 使用softmax
prob = tf.nn.softmax(logits,axis=1) #对整个列表的值求softmax
print(prob)
print('=================================================================================================================================')

# 求和
print(tf.reduce_sum(prob,axis=1))
 
# logits：array([[-0.80400324,  1.1189251 ,  1.7390656 ,  0.8317995 , -1.1970901 ,-0.48806524, -1.1161256 , -0.35890388,  0.765254  , -1.8393846 ]],dtype=float32)>
# prob：array([[0.02841684, 0.19439851, 0.3614236 , 0.14588003, 0.01918051,0.03897498, 0.02079806, 0.0443486 , 0.13648833, 0.01009056]],dtype=float32)>
# sum：<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>


tf.Tensor(
[[ 1.788518   -0.10648251 -1.6808982  -1.3044801  -1.042417    0.53260565
  -0.28296232  0.30348158 -0.09753752 -0.51368237]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[0.45981377 0.06911841 0.01431639 0.02085979 0.02710956 0.13096227
  0.05793614 0.10414511 0.06973945 0.04599907]], shape=(1, 10), dtype=float32)
tf.Tensor([1.], shape=(1,), dtype=float32)


In [14]:
"""
2.3 tanh函数
tanh是通过sigmoid函数放大平移得到，输出结果在(-1,1)之间

tanh函数： tf.tanh(tensor)
"""
a = tf.linspace(-2, 2, 5)  # -2到2的长度为5的向量
print(a)
print('=================================================================================================================================')

# 结果映射到(-1,1)
print(tf.tanh(a))
# 输入：<tf.Tensor: shape=(5,), dtype=float64, numpy=array([-2., -1.,  0.,  1.,  2.])>
# 输出：<tf.Tensor: shape=(5,), dtype=float64, numpy=array([-0.96402758, -0.76159416,  0.        ,  0.76159416,  0.96402758])>


tf.Tensor([-2. -1.  0.  1.  2.], shape=(5,), dtype=float64)
tf.Tensor([-0.96402758 -0.76159416  0.          0.76159416  0.96402758], shape=(5,), dtype=float64)


In [15]:
"""
2.4 relu函数
将输入小于0的值都变成0，大于0的值保持不变。

relu激活函数：tf.nn.relu(features)
公式为： f(x)=\begin{bmatrix} 0\;\,\;x<0 & \\ x \;\;\;x>=0 & \end{bmatrix}
"""
# relu函数，x小于0，梯度0，x大于0梯度1
a = tf.linspace(-1,1,10) # -1到1，长度为10的向量
print(a)
print('=================================================================================================================================')

print(tf.nn.relu(a))
 
# 输入为：array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])>
# 输出为：array([0.        , 0.        , 0.        , 0.        , 0.        ,       0.11111111, 0.33333333, 0.55555556, 0.77777778, 1.        ])>

tf.Tensor(
[-1.         -0.77777778 -0.55555556 -0.33333333 -0.11111111  0.11111111
  0.33333333  0.55555556  0.77777778  1.        ], shape=(10,), dtype=float64)
tf.Tensor(
[0.         0.         0.         0.         0.         0.11111111
 0.33333333 0.55555556 0.77777778 1.        ], shape=(10,), dtype=float64)


In [16]:
"""
2.5 leaky_relu函数
非饱和激活函数： tf.nn.leaky_relu( features, alpha=0.2 )
features： tensor类型，表示预激活值，必须是下列类型之一：float16,float32,float64,int32,int64.
alpha： x <0时激活函数的斜率.
由于relu函数对于小于0的值，这个神经元的梯度永远都会是0，在实际操作中，如果数据很大，很可能网络中较多的神经元都死了。这时需要leaky_relu函数：
公式：y = max(0, x) + leak*min(0,x) 
leak是一个很小的常数，这样保留了一些负轴的值，使得负轴的信息不会全部丢失
"""
a = tf.linspace(-1,1,10)
print(a)
print('=================================================================================================================================')

# tf.nn.leaky_rulu() ，x小于0y也小于0但接近0
print(tf.nn.leaky_relu(a))
 
# 输入：array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])>
# 输出：array([-0.2       , -0.15555556, -0.11111111, -0.06666667, -0.02222222,        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])>

tf.Tensor(
[-1.         -0.77777778 -0.55555556 -0.33333333 -0.11111111  0.11111111
  0.33333333  0.55555556  0.77777778  1.        ], shape=(10,), dtype=float64)
tf.Tensor(
[-0.2        -0.15555556 -0.11111111 -0.06666667 -0.02222222  0.11111111
  0.33333333  0.55555556  0.77777778  1.        ], shape=(10,), dtype=float64)
